### Question search engine

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.4-py3-none-any.whl size=

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/70.8M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/7.83M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
model = model.to(device)

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (2 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=32,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

In [ ]:
from torch.amp import autocast
from tqdm import tqdm
total_samples = 0
correct_predictions = 0

with torch.no_grad():
    for batch in tqdm(val_loader, desc='Validation'):
        with autocast(device_type=device.type, dtype=torch.float16):
            predicted = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device)
            )
        labels = batch['labels'].to(device)
        logits = predicted.logits
        predictions = torch.argmax(torch.softmax(logits, dim=1), dim=1)  # Get predicted class
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)



#print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Validation: 100%|██████████| 1264/1264 [01:10<00:00, 18.06it/s]


__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
#<A whole lot of YOUR CODE HERE>
accuracy = correct_predictions/total_samples
accuracy

0.9084095968340341

In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (8 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

I chose the [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base) model:

In [ ]:
#<A whole lot of your code goes here>
model = transformers.AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base")
model = model.to(device)

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base):

In [ ]:
#Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result
qqp_preprocessed = qqp.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
# Training
train_set = qqp_preprocessed['train']
train_set_reduced = train_set.shuffle(seed=42).select(range(int(len(train_set) * 0.2)))
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=64,
    shuffle=True,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

#Validation
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=64,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

In [ ]:
from torch.amp import GradScaler, autocast
from tqdm import tqdm

loss_fct = nn.CrossEntropyLoss()
scaler = GradScaler()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model.train()
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        optimizer.zero_grad()

        # Forward pass with mixed precision
        with autocast(device_type=device.type, dtype=torch.float16):
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device)
            )
            labels = batch['labels'].to(device)
            loss = loss_fct(outputs.logits, labels)

        # Backward pass and optimization
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        #Free up memory
        del labels, outputs
        torch.cuda.empty_cache()

        #Update progress bar
        progress_bar.set_postfix(loss=loss.item())

Epoch 1/1


Training: 100%|██████████| 5686/5686 [1:07:58<00:00,  1.39it/s, loss=0.125]


Testing the model accuracy:

In [ ]:
model.eval()
total_samples = 0
correct_predictions = 0

with torch.no_grad():
    for batch in tqdm(val_loader, desc='Validation'):
        with autocast(device_type=device.type, dtype=torch.float16):
            predicted = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device)
            )
        labels = batch['labels'].to(device)
        #break
        logits = predicted.logits
        predictions = torch.argmax(torch.softmax(logits, dim=1), dim=1)  # Get predicted class
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

Validation: 100%|██████████| 632/632 [01:54<00:00,  5.52it/s]


In [ ]:
accuracy = correct_predictions/total_samples
accuracy

0.8977986643581499

Save the model:

In [ ]:
torch.save(model.state_dict(), f'llm_model_acc_{str(round(accuracy*100, 2)).replace(".","-")}.pt')

Notes:
1. Larger batch leads to GPU allocation memory problems (run out of GPU memory)
2. autocast for optimization
3. training for 1 epoch with full training dataset (too long for more, even with gpu) gave an accuracy comparable to the previous model: around `0.898`.

### Task 3: try the full pipeline (4 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

Let's start. First, I load an existing best model if found:

In [ ]:
load_model = True
model_url = 'https://drive.google.com/file/d/1GiO06TMJEIMQgx1OD0CQNPQMAZi-jLt7/view?usp=sharing'
model_url = 'https://drive.google.com/uc?id='+model_url.split('/')[-2]

In [ ]:
import os
from copy import deepcopy
import gdown

if load_model:
    gdown.download(model_url, 'best_llm-model_state.pt', quiet=False)
    #!wget -O 'best_model_state.pt' '{model_url}'
    if os.path.exists('best_llm-model_state.pt'):
        print('Loading the best model...')
        model = transformers.AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base").to(device)
        model_dict = torch.load('best_llm-model_state.pt', weights_only=True, map_location=torch.device(device))
        llm_model = deepcopy(model).to(device)
        llm_model.load_state_dict(model_dict)
        print('Best model loaded.')

Downloading...
From (original): https://drive.google.com/uc?id=1GiO06TMJEIMQgx1OD0CQNPQMAZi-jLt7
From (redirected): https://drive.google.com/uc?id=1GiO06TMJEIMQgx1OD0CQNPQMAZi-jLt7&confirm=t&uuid=1869e392-08db-419a-803d-a7540783b45e
To: /content/best_llm-model_state.pt
100%|██████████| 738M/738M [00:10<00:00, 69.0MB/s]


Loading the best model...


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Best model loaded.


In [ ]:
# Training
train_set = qqp_preprocessed['train']
train_set_reduced = train_set.shuffle(seed=42).select(range(int(len(train_set) * 0.2)))
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=64,
    shuffle=True,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

#Validation
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=64,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

In [ ]:
from torch.amp import GradScaler, autocast
from tqdm import tqdm
llm_model.eval()
total_samples = 0
correct_predictions = 0


with torch.no_grad():
    for batch in tqdm(val_loader, desc='Validation'):
        with autocast(device_type=device.type, dtype=torch.float16):
            predicted = llm_model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device)
            )
        labels = batch['labels'].to(device)
        #break
        logits = predicted.logits
        predictions = torch.argmax(torch.softmax(logits, dim=1), dim=1)  # Get predicted class
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)
accuracy = correct_predictions/total_samples
accuracy

Validation: 100%|██████████| 632/632 [01:59<00:00,  5.30it/s]


0.37031906999752656

Time to test the fine-tuned [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base) model: find duplicate questions

In [ ]:
def encode_question(question, model=llm_model):
    inputs = tokenizer(question, padding='max_length', max_length=128, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.cpu().numpy()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

The straightforward solution:

In [ ]:
def find_top_duplicates_old(question, train_dataset, tokenizer, model=llm_model, top_k=5):
    # Encode the input question
    inputs_emb = encode_question(question)
    similarities = []
    for example in tqdm(train_dataset, desc='Searching'):
        # Pass the example through the model to encode it
        example_emb = encode_question(example['text1'])

        # Compute cosine similarity of the two encodings
        sim = cosine_similarity(inputs_emb, example_emb)
        similarities.append((sim, example['text1']))

    # Sort similarities in descending order
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)

    # Get the top-K most similar questions
    top_duplicates = similarities[:top_k]
    return top_duplicates

A faster solution to find duplicate questions, using `TF-IDF` vectorisation:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

# Use lightweight embedding to generate a TF-IDF representation for all dataset questions
def create_tfidf_vectorizer(train_dataset):
    texts = [example['text1'] for example in train_dataset]
    vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(texts)
    return vectorizer, tfidf_matrix

# Find top candidates using TF-IDF
def find_initial_candidates(question, tfidf_vectorizer, tfidf_matrix, top_k=100):
    question_vector = tfidf_vectorizer.transform([question])
    similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()

    # Get top-k indices
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return top_indices, similarities[top_indices]

# Update find_top_duplicates
def find_top_duplicates(question, train_dataset, tokenizer, model=llm_model, top_k=5):
    # Create TF-IDF representation
    vectorizer, tfidf_matrix = create_tfidf_vectorizer(train_dataset)

    # Find initial candidates
    initial_indices, initial_scores = find_initial_candidates(question, vectorizer, tfidf_matrix)

    # Encode the input question
    inputs_emb = encode_question(question)
    similarities = []

    # Calculate similarities only for top candidates
    for idx in initial_indices:
        example_emb = encode_question(train_dataset[int(idx)]['text1'])
        sim = cosine_similarity(inputs_emb, example_emb)
        similarities.append((sim, train_dataset[int(idx)]['text1']))

    # Sort for top-K results
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
    return similarities[:top_k]

In [ ]:
questions = [
    "How can I learn Python programming?",
    "What is the best way to improve my skating skills?",
    "What are the benefits of fried chicken?",
    "How do I become a pirate?",
    "What is the best job nowadays?",
]

# Showcase for 5 examples
for question in questions:
    print(f"\nInput Question: {question}")
    top_results = find_top_duplicates(question, qqp_preprocessed['train'], tokenizer)
    for rank, (score, duplicate) in enumerate(top_results):
        print(f"Top {rank+1} (Score: {score[0][0]:.4f}): {duplicate}")


Input Question: How can I learn Python programming?
Top 1 (Score: 1.0000): How can I learn C#?
Top 2 (Score: 0.9998): What can I do on python?
Top 3 (Score: 0.9998): What can I do on python?
Top 4 (Score: 0.9998): What can I do on python?
Top 5 (Score: 0.9998): What can I do on python?

Input Question: What is the best way to improve my skating skills?
Top 1 (Score: 1.0000): What is the best way to improve my questions on Quora?
Top 2 (Score: 1.0000): What is the best way to improve my questions on Quora?
Top 3 (Score: 1.0000): What is the best way to improve my questions on Quora?
Top 4 (Score: 1.0000): What is the best way to improve my questions on Quora?
Top 5 (Score: 1.0000): What is the best way to improve my questions on Quora?

Input Question: What are the benefits of fried chicken?
Top 1 (Score: 1.0000): What are the benefits of reciting Hanuman Chalisa?
Top 2 (Score: 1.0000): What are some of the best ways to quart a chicken?
Top 3 (Score: 1.0000): What are the benefits of s